In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

# FCN

In [2]:
class NN(nn.Module):
    def __init__(self, input_size, num_classes):
        super(NN, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, num_classes)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [3]:
model = NN(784, 10)
x = torch.randn(64, 784)
model(x).shape

torch.Size([64, 10])

# Settings

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

input_size = 784
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 1

# Load Data

In [5]:
train_dataset = datasets.MNIST(
    root='dataset/',
    train=True, 
    transform=transforms.ToTensor(), 
    download=True
)

test_dataset = datasets.MNIST(
    root='dataset/',
    train=False, 
    transform=transforms.ToTensor(), 
    download=True
)

In [6]:
train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True
)

test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=batch_size,
    shuffle=True
)

# Initalize

In [7]:
model = NN(
    input_size=input_size,
    num_classes=num_classes,
).to(device)
model

NN(
  (fc1): Linear(in_features=784, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train

In [9]:
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(train_loader):
        
        # cuda
        data = data.to(device)
        targets = targets.to(device)
        
        # flatten
        data = data.reshape(data.shape[0], -1)
        
        # forward
        scores = model(data)
        loss = criterion(scores, targets)
        
        # backward
        optimizer.zero_grad()
        loss.backward()
        
        # gradient descent
        optimizer.step()
    
    print(f'epoch: {epoch} / {num_epochs}')

epoch: 0 / 1


In [10]:
def check_acc(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()
    
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)
            x = x.reshape(x.shape[0], -1)
            
            scores = model(x)
            _, pred = scores.max(1)
            num_correct += (pred == y).sum()
            num_samples += pred.size(0)
        
        print(f'{num_correct} / {num_samples}, acc: {float(num_correct) / float(num_samples) * 100:.2f}')

In [11]:
check_acc(train_loader, model)

55664 / 60000, acc: 92.77


In [12]:
check_acc(test_loader, model)

9287 / 10000, acc: 92.87
